In [ ]:
# 구글 번역 api (처음 한번만 설치)
%pip install googletrans==4.0.0-rc1

In [39]:
# 이 파일 있는 디렉터리로 이동??
import os
os.chdir(os.getcwd())
%pwd

#%cd /content/drive/MyDrive/Github/floread/sentiment-analysis

'c:\\Users\\KangIW\\git_j500\\floread'

In [9]:
# 가장 퀄리티가 나은 감성대화말뭉치 사용
import pandas as pd

df = pd.read_csv('data/감성대화말뭉치(병합).csv',index_col=0)
df.shape

(58240, 2)

In [11]:
df['emotion'] = df['emotion'].replace('상처', '슬픔')
df.groupby('emotion').count().iloc[:,0]

emotion
기쁨     7337
당황     9797
분노    10414
불안    10429
슬픔    20263
Name: sentence, dtype: int64

In [15]:
import re

#한글 있는지 체크
def has_korean(text):
    korean_check = lambda text: any(char >= '가' and char <= '힣' for char in text)
    return korean_check(text)

#0으로 끝나는 문장
def remove_end_0(sentence):
    if sentence.endswith('0'):
        return sentence[:-1]
    return sentence

def Preprocess_all(df):
    
    print("---------------전처리 전: ",df.shape)
    
    # 결측치 제거
    df = df.dropna()
    
    #필터 처리된 단어 공백으로 치환
    df['sentence'] = df['sentence'].str.replace(r'OO+', ' ', regex=True)
    
    #좌측 공백 제거
    df['sentence'] = df['sentence'].str.lstrip()
    
    #한글 없는 문장 제거
    df = df[df['sentence'].apply(lambda x: has_korean(x))]
    
    #한자 들어간 문장 제거
    df = df[~df['sentence'].str.contains('[\u4e00-\u9fff]')]
        
    #일부 단어 치환
    df['sentence'] = df['sentence'].str.replace(r'(ㅋ{3,})', 'ㅋㅋ', regex=True)
    df['sentence'] = df['sentence'].str.replace(r'\?\?{3,}', '??,', regex=True)
    df['sentence'] = df['sentence'].str.replace(r'~{3,}', '~~', regex=True)
    df['sentence'] = df['sentence'].str.replace(r'!{3,}', '!!', regex=True)
    df['sentence'] = df['sentence'].str.replace(r'-{3,}', '--', regex=True)
    df['sentence'] = df['sentence'].str.replace(r'(ㅠ{3,})', 'ㅠㅠ', regex=True)
    df['sentence'] = df['sentence'].str.replace(r'(ㅜ{3,})', 'ㅜㅜ', regex=True)
    df['sentence'] = df['sentence'].str.replace(r'(ㄱ{3,})', 'ㄱㄱ', regex=True)
    df['sentence'] = df['sentence'].str.replace(r'(ㄷ{3,})', 'ㄷㄷ', regex=True)
    df['sentence'] = df['sentence'].str.replace(r'(ㅎ{3,})', 'ㅎㅎ', regex=True)

    df['sentence'] = df['sentence'].str.replace(r'(♥{2,})', '♥', regex=True)
    df['sentence'] = df['sentence'].str.replace(r'(♡{2,})', '♡', regex=True)
    df['sentence'] = df['sentence'].str.replace(r'\s{2,}', ' ', regex=True)
    
    df['sentence'] = df['sentence'].str.replace(r'\.{7,}', '......', regex=True)
    df['sentence'] = df['sentence'].str.replace(';{3,}', ';;')
    df['sentence'] = df['sentence'].str.replace('\^{3,}', '^^')
    
    # 한글 자모음이 3번 이상 반복되는 패턴을 찾아, 그 자모음 2개로 치환
    df['sentence'] = df['sentence'].apply(lambda x: re.sub(r'(([ㄱ-ㅎㅏ-ㅣ]|ㄲ|ㄸ|ㅃ|ㅆ|ㅉ){3,})', lambda m: m.group(1)[:2], x))

    # 길이가 100이상인 문장 제거
    df = df[df['sentence'].str.len() < 100]
    
    # 0으로 끝나면 0빼기
    df['sentence'] = df['sentence'].apply(remove_end_0)
    
    #의미 없는 0패턴 제거
    replacements = {
        r'\?0': '?',
        r';0': ';',
        r'ㅠ0': 'ㅠ',
        r'ㅜ0': 'ㅜ',
        r'\.0': '.'
    }

    for pattern, replacement in replacements.items():
        df['sentence'] = df['sentence'].str.replace(pattern, replacement, regex=True)
        
    # 중복행 제거
    dup = df[df.duplicated(subset=['sentence'])]
    print("중복되는 행: ", len(dup))
    df = df.drop_duplicates(subset=['sentence'], keep='first')
    
    # 인덱스 재설정
    df.reset_index(drop=True, inplace=True)

    print("---------------전처리 후: ",df.shape)    
    
    return df

In [16]:
pp_df = Preprocess_all(df)

---------------전처리 전:  (58240, 2)
중복되는 행:  0
---------------전처리 후:  (58224, 2)


In [ ]:
pp_df.to_csv('data/감성대화말뭉치_전처리완.csv', encoding='utf-8-sig')

In [2]:
df = pd.read_csv('data/감성대화말뭉치_전처리완.csv',index_col=0)
df.groupby('emotion').count().iloc[:,0]

NameError: name 'pd' is not defined

In [28]:
dfsum = pd.read_csv('data/병합데이터셋-v3.csv',index_col=0)
print(dfsum.shape)
dfsum.groupby('emotion').count().iloc[:,0]

(135791, 2)


emotion
놀람    19538
분노    19526
불안    15980
슬픔    21305
중립    45149
행복    14293
Name: sentence, dtype: int64

놀람1000,분노1000, 불안4000, 슬픔 -, 중립-, 행복6000

In [3]:
import pandas as pd
df = pd.read_csv('data/감성대화말뭉치_전처리완.csv',index_col=0)
df = df[df['emotion'] != '슬픔']
df.shape

(37966, 2)

In [4]:
from googletrans import Translator
import time

# 전역으로 선언하면 오버해드가 줄어든다?
translator = Translator()

# 역번역 함수: 한글 -> 영어 -> 한글
def backtranslate(text):
    global translator   
    
    en = translator.translate(text, src='ko', dest='en').text
    time.sleep(0.5)   #ReadTimeout 방지
    
    kr = translator.translate(en, src='en', dest='ko').text
    time.sleep(0.25)   
    
    return kr

In [12]:
# 카운트 테스트
from googletrans import Translator
import time

translator = Translator()

cnt=0
# 역번역 함수: 한글 -> 영어 -> 한글
def backtranslate(text):
    global translator   
    global cnt
    
    cnt +=1
    if cnt%100 ==0: print(cnt)
    en = translator.translate(text, src='ko', dest='en').text
    time.sleep(0.3)   #ReadTimeout 방지
    
    kr = translator.translate(en, src='en', dest='ko').text
    time.sleep(0.3)   
    
    return kr

In [10]:
# (에러방지)데이터 프레임 분리 후 각각 번역
df1 = df[:3000]
df2 = df[3000:6000]
df3 = df[6000:9000]
df4 = df[9000:12000]
df5 = df[12000:15000]
df6 = df[15000:18000]
df7 = df[18000:21000]
df8 = df[21000:24000]
df9 = df[24000:27000]
df10 = df[27000:30000]
df11 = df[30000:33000]
df12 = df[33000:36000]
df13 = df[36000:]

In [13]:
df1['sentence'] = df1['sentence'].apply(backtranslate)
df1.to_csv('res/df1.csv', encoding='utf-8-sig')

ConnectTimeout: timed out

In [ ]:
df2['sentence'] = df1['sentence'].apply(backtranslate)
df2.to_csv('res/df1_bt.csv', encoding='utf-8-sig')